In [24]:
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time

def get_by_css(session, css, text=True):
    try:
        MAX_TRY = 3
        t = 0
        while t < MAX_TRY:
            t += 1
            try:
                res = session.find_element_by_css_selector(css)
            except StaleElementReferenceException:
                time.sleep(2)
                continue
            break
        if t == MAX_TRY:
            return None
    except NoSuchElementException:
        return None
    if text:
        return res.text
    return res
#I find that some hotels are new opened so they do not have score and comments,the elements can not be located and the error message is NoSuchElementException. So I use try except.
#resource:https://www.cnblogs.com/yoyoketang/p/6696788.html

In [25]:
def get_hotels_from_browser(b):

    hotels = []
    for session in b.find_elements_by_css_selector('.sr_item.sr_item_new.sr_item_default.sr_property_block.sr_flex_layout'):
        hotel = {}
        name = get_by_css(session, 'span.sr-hotel__name')
        score = get_by_css(session, '.bui-review-score__badge')
        comments = get_by_css(session, '.bui-review-score__text')
        address = get_by_css(session, '.jq_tooltip.district_link.visited_link')
        price = get_by_css(session, '.price.availprice.no_rack_rate')
        if not price:
            price = get_by_css(session, '.price.scarcity_color')

        hotel['name'] = name
        hotel['score'] = score
        hotel['comments'] = comments
        hotel['address'] = address
        hotel['price'] = price
        value_list=list(hotel.values())
        hotels.append(value_list)
    return hotels

In [26]:
url = 'https://www.booking.com/searchresults.zh-cn.html?aid=378266&label=booking-name-IquAp*EbiLS6jPVl_he8yQS267778093594%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap1t1%3Aneg%3Afi%3Atikwd-65526620%3Alp9061634%3Ali%3Adec%3Adm&sid=75a6a10fd83d509dc44f2bc75e94dc31&checkin_month=9&checkin_monthday=12&checkin_year=2019&checkout_month=9&checkout_monthday=13&checkout_year=2019&city=-240905&class_interval=1&dest_id=-240905&dest_type=city&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=city&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&srpvid=342a6023a36e0036&ss=%E5%A4%A7%E9%98%AA&ssb=empty&ssne=%E5%A4%A7%E9%98%AA&ssne_untouched=%E5%A4%A7%E9%98%AA&nflt=di%3D7322%3Bpri%3D2%3B&rsf=pri-2&lsf=pri%7C2%7C136'
browser = webdriver.Chrome()
browser.get(url)
time.sleep(2)

In [27]:
all_page_hotels = []
for i in range(7):
    time.sleep(2)
    new_hotels= get_hotels_from_browser(browser)
    all_page_hotels.extend(new_hotels)
    time.sleep(0.5)
    no_survey = get_by_css(browser, '.survey_no_button', False)
    if no_survey:
        no_survey.click()
    time.sleep(1)
    next_button = browser.find_element_by_css_selector('.bui-pagination__nav ul.bui-pagination__list li.bui-pagination__item.bui-pagination__next-arrow a') #get the element's location
    next_button.location
    loc = next_button.location
    browser.execute_script("window.scrollTo({x}, {y});".format(**loc))
    time.sleep(2)
    next_button.click()

print(all_page_hotels)
#In this part I add some time.sleep() to solve the browser rendering delay, resource:https://github.com/hupili/python-for-data-and-media-communication-gitbook/issues/68


[['Grandi Nipponbashi Park Hotel', '8.5', '244条住客点评', '南部地区, 大阪 – 显示在地图上', 'HK$ 582'], ['Kawarayamachi Crystal Hotel', '8.7', '299条住客点评', '南部地区, 大阪 – 显示在地图上', 'HK$ 588'], ['Floral Hotel Shinsaibasi', '7.6', '59条住客点评', '南部地区, 大阪 – 显示在地图上', 'HK$ 726'], ['Namba~Tsutenkaku Guest house Sunnyhouse', '9.1', '16条住客点评', '南部地区, 大阪 – 显示在地图上', 'HK$ 657'], ['Shin Nippombashi', '9.0', '68条住客点评', '南部地区, 大阪 – 显示在地图上', 'HK$ 807'], ['Doutonbori Crystal Hotel', '9.5', '5条住客点评', '道顿堀, 大阪 – 显示在地图上', 'HK$ 872'], ['RakutenSTAYHOSTEL Nambaminami', '8.5', '127条住客点评', '南部地区, 大阪 – 显示在地图上', 'HK$ 795'], ['Lahaina Park Hotel', '7.6', '9条住客点评', '南部地区, 大阪 – 显示在地图上', 'HK$ 553'], ['APA Hotel Namba-Eki Higashi', '8.2', '5,355条住客点评', '难波, 大阪 – 显示在地图上', 'HK$ 761'], ['Hotel Shin-Imamiya', '7.1', '3,211条住客点评', '南部地区, 大阪 – 显示在地图上', 'HK$ 552'], ['Jakotel Apartment Namba', '7.8', '128条住客点评', '南部地区, 大阪 – 显示在地图上', 'HK$ 484'], ['HG Cozy Hotel No.16', '7.3', '30条住客点评', '南部地区, 大阪 – 显示在地图上', 'HK$ 616'], ['APA Hotel Namba-Shinsaibash

In [28]:
import csv
with open('osaka.csv','w',newline='')as f:
    mywriter = csv.writer(f)
    header=['name','score','comments','address','price']
    mywriter.writerow(header)
    mywriter.writerows(all_page_hotels)